## Author's notes
### 28.10.2025
I am starting the model creation process. The files created in the prerocessing will be used, however, I do have some issues with them.
1) The data has not been split into train and test set during the preprocessing phase. I am aware that label encoding of categorical variables has been done and we should know all the categories for the teams, but it is still best practice to do encoding AFTER the split in order to avoid data leakage and it may be a problem in the assessment of our work by the teachers. Therefore, it would probably be for the best if this was corrected. All the preprocessing steps should be fitted to only the train data and only then used to transform the test data.
2) Only the market values were used in the bigger dataset. I am not saying this is wrong or right and I will trust Vojta on this one, BUT there should be a very detailed explanation for why exactly we didn't use the rest of the data.

Because of the issues with the train test split, I will split the data here, but I would like to change it once it is fixed.